In [1]:
import pandas as pd

In [2]:
original_data = pd.read_csv("Pokemon.csv")

In [3]:
original_data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [4]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


In [5]:
original_data.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [8]:
features = original_data.columns.tolist()[4:len(original_data.columns.tolist())-1]
features

['Total',
 'HP',
 'Attack',
 'Defense',
 'Sp. Atk',
 'Sp. Def',
 'Speed',
 'Generation']

In [10]:
target = "Legendary"

In [13]:
X = original_data[features]
X

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
0,318,45,49,49,65,65,45,1
1,405,60,62,63,80,80,60,1
2,525,80,82,83,100,100,80,1
3,625,80,100,123,122,120,80,1
4,309,39,52,43,60,50,65,1
...,...,...,...,...,...,...,...,...
795,600,50,100,150,100,150,50,6
796,700,50,160,110,160,110,110,6
797,600,80,110,60,150,130,70,6
798,680,80,160,60,170,130,80,6


In [14]:
y = original_data[target]
y

0      False
1      False
2      False
3      False
4      False
       ...  
795     True
796     True
797     True
798     True
799     True
Name: Legendary, Length: 800, dtype: bool

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(640, 8) (640,)
(160, 8) (160,)


훈련데이터 640개, 테스트 데이터 640개 나누어짐

In [17]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=25)

In [19]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [20]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[144,   3],
       [  5,   8]], dtype=int64)

##### 이 데이터에서는 Positive는 Legendary=True, Negative는 Legendary=False

---

#### 왼쪽 위에서부터 순서대로 TN,FP,FN,TP

- TN (True Negative): 옳게판단한 Negative, 일반포켓몬을 일반포켓몬이라고 알맞게 판단한 경우
- FP (False Positive): 틀리게 판단한 Positive, 일반포켓몬을 전설포켓몬이라고 판단한 경우
- FN (False Negative): 틀리게 판단한 Negative, 전설포켓몬을 일반포켓몬이라고 판단한 경우
- TP (True Positive): 옳게판단한 Positive, 전설포켓몬을 전설포켓몬이라고 알맞게 판단한 경우

In [28]:
# 위의 confusion_matrix로 정확도 간단하게 계산하기
print(f"{152 / 160 * 100}%") # 맞은 데이터갯수 / 전체 데이터 갯수

95.0%


In [29]:
len(original_data[original_data["Legendary"] == True])

65

위의 정확도가 불확실한 이유가 전체데이터의 불균형때문, 전체 800마리중 단 65마리만 전설, 나머지 735가 일반이면
모델 훈련필요없이 735/800*100 = 92% 의 정확도가 나옴 그래서 정확도가 아닌 다른 척도로 모델의 성능을 평가해야함

In [31]:
# classification_report를 활용해서 다른 수치들도 볼수 있음
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.97      0.98      0.97       147
        True       0.73      0.62      0.67        13

    accuracy                           0.95       160
   macro avg       0.85      0.80      0.82       160
weighted avg       0.95      0.95      0.95       160

